In [20]:
import pandas as pd
from pymer4.models import Lmer

from load_feat_pd import *
import matplotlib.pyplot as plt
import seaborn as sns
# import rpy2.robjects as ro
# from rpy2.robjects import pandas2ri
# from rpy2.robjects.packages import importr
# from rpy2.robjects import Formula

def lme(metadata, outfile, featname='shimmer', level='utt', stats='mean' , exp_interaction=False):

    outfile, gender_ref = outfile
    df = pd.DataFrame(metadata)
    
    # remove group id == '11':
    df = df[df['group_id'] != '11']
    
    # map group_id 21 to HC 22 to PD
    df['group_id'] = df['group_id'].map({'21': 'HC', '22': 'PD'})
    df['group_id'] = df['group_id'].astype('category')


    # change age from string to int
    df['age'] = pd.to_numeric(df['age'], errors='coerce')
    missing_age = df['age'].isnull().sum()
    if missing_age > 0:
        print(f"warning 'age' has {missing_age} NaN, has been deleted")
        df = df.dropna(subset=['age'])
        
    # add one more column item name in the filename
    df['item'] = df['filename'].apply(lambda x: x.split('.')[0].split('_')[-1])
    
    # df['group_id'] = pd.Categorical(df['group_id'], categories=['PD', 'Control'])
    if gender_ref == 'V':
        df['gender'] = pd.Categorical(df['gender'], categories=['V', 'M'])
    df['experiment'] = pd.Categorical(df['experiment'])
    # df['experiment'] = pd.Categorical(df['experiment'], categories=['exp_1_PictureNaming', 'exp_2_EarlyLate', 'exp_3_BoundaryTone'])

    # print("Group ID Levels:", df['group_id'].cat.categories)       # 输出: ['PD', 'Control']
    # print("Gender Levels:", df['gender'].cat.categories)           # 输出: ['Female', 'Male']
    # print("Experiment Levels:", df['experiment'].cat.categories)   # 输出: ['Baseline', 'EarlyLate', 'BoundaryTone']

    print(df.head())


    if level == 'utt':
        # change the value from np array to float 
     
        if type(df['value'].iloc[0]) == np.ndarray:
            df['value'] = df['value'].apply(lambda x: float(x))
        
    elif level == 'frame':
        if stats == 'mean':
            df['value'] = df['value'].apply(lambda x: x.mean())
        elif stats == 'std':
            df['value'] = df['value'].apply(lambda x: x.std())
        elif stats == 'median':
            df['value'] = df['value'].apply(lambda x: np.median(x))

    # print(df.head())
    # print(df.shape)
    featname = "response_time" if featname == 'rp' else featname
    valuetype = stats if level == 'frame' else ''

    print(f'=============== value: {featname}_{valuetype} ====================\n')

    # Activate pandas to R DataFrame conversion
    # pandas2ri.activate()

    # # Import necessary R packages
    # base = importr('base')
    # utils = importr('utils')

    # stats = importr('stats')
    # lme4 = importr('lme4')
    # emmeans = importr('emmeans')


    # lmerTest = importr('lmerTest')

    # df_r = pandas2ri.py2rpy(df)
    if exp_interaction:
        model = Lmer("value ~ age + group_id*experiment + group_id*gender + (1|subject_id) + (1|item)", data=df)
    else:
        model = Lmer("value ~ age + experiment + group_id*gender + (1|subject_id) + (1|item)", data=df)

    result = model.fit()
    # model_formula = Formula("value ~ age + group_id*experiment + group_id*gender + (1|subject_id) + (1|item)")


    # # Assign the formula to R's environment
    # ro.globalenv['model_formula'] = model_formula

    # # Assign the data to R's environment
    # ro.globalenv['df_r'] = df_r

    # model_r = lmerTest.lmer(ro.globalenv['model_formula'], data=ro.globalenv['df_r'])
    # ro.globalenv['model_r'] = model_r

    # # # Get model summary
    # # summary = ro.r('summary(model_r)')
    # # print(summary)
    # summary = ro.r('capture.output(summary(model_r))')
    # print(len(list(summary)))
    # for i in range(len(list(summary))):
    #     print('=========================={}=========================='.format(i))
    #     print(list(summary)[i])
        
        


        
    with open(outfile, 'a') as f:
        f.write(f'\n')
        f.write(f'=============== value: {featname}_{valuetype} ====================\n')
        f.write(result[['Estimate', 'SE', 'T-stat', 'P-val', 'Sig']].to_string())    
        f.write(f'\n')
        f.write(f'\n')
        f.write(f'\n')
    
    return df

def lme_cmp_gender(metadata, outfile, featname='shimmer', level='utt', stats='mean' , exp_interaction=False):
    
    outfile, gender_ref = outfile
    df = pd.DataFrame(metadata)
    
    # remove group id == '11':
    df = df[df['group_id'] != '11']
    
    # map group_id 21 to HC 22 to PD
    df['group_id'] = df['group_id'].map({'21': 'HC', '22': 'PD'})
    df['group_id'] = df['group_id'].astype('category')

    # change age from string to int
    df['age'] = pd.to_numeric(df['age'], errors='coerce')
    missing_age = df['age'].isnull().sum()
    if missing_age > 0:
        print(f"warning 'age' has {missing_age} NaN, has been deleted")
        df = df.dropna(subset=['age'])
        
    # add one more column item name in the filename
    df['item'] = df['filename'].apply(lambda x: x.split('.')[0].split('_')[-1])
            
    df['experiment'] = pd.Categorical(df['experiment'])

    if level == 'utt':
        if type(df['value'].iloc[0]) == np.ndarray:
            df['value'] = df['value'].apply(lambda x: float(x))
        
    elif level == 'frame':
        if stats == 'mean':
            df['value'] = df['value'].apply(lambda x: x.mean())
        elif stats == 'std':
            df['value'] = df['value'].apply(lambda x: x.std())
        elif stats == 'median':
            df['value'] = df['value'].apply(lambda x: np.median(x))

    featname = "response_time" if featname == 'rp' else featname
    valuetype = stats if level == 'frame' else ''

    # print(f'=============== value: {featname}_{valuetype} ====================\n')

    if exp_interaction:
        model = Lmer("value ~ age + group_id*experiment + group_id*gender + (1|subject_id) + (1|item)", data=df)
    else:
        model = Lmer("value ~ age + experiment + group_id*gender + (1|subject_id) + (1|item)", data=df)

    result = model.fit()
    
    interaction = result.iloc[[-1]]
    
    
    group_effect_male = result.iloc[[-3]]
    
    df['gender'] = pd.Categorical(df['gender'], categories=['V', 'M'])

    if exp_interaction:
        model = Lmer("value ~ age + group_id*experiment + group_id*gender + (1|subject_id) + (1|item)", data=df)
    else:
        model = Lmer("value ~ age + experiment + group_id*gender + (1|subject_id) + (1|item)", data=df)

    result = model.fit()
    
    group_effect_female = result.iloc[[-3]]
    
    sig_interaction = interaction['Sig'].values[0]
    sig_group_effect_male = group_effect_male['Sig'].values[0]
    sig_group_effect_female = group_effect_female['Sig'].values[0]
    
    if sig_interaction == '' and sig_group_effect_male == '' and sig_group_effect_female == '':
        return df
    
    if sig_interaction != '':
        with open(outfile + '_interaction.txt', 'a') as f:
            f.write(f'{featname}_{valuetype}|{sig_interaction}')
            f.write(f'\n')

            
    if sig_group_effect_male != '' and sig_group_effect_female != '':
        with open(outfile + '_SigInBothGender.txt', 'a') as f:
            f.write(f'{featname}_{valuetype}|{sig_group_effect_male}|{sig_group_effect_female}')
            f.write(f'\n')


    if sig_group_effect_male != '':
        with open(outfile + '_SigOnlyInMale.txt', 'a') as f:
            f.write(f'{featname}_{valuetype}|{sig_group_effect_male}')
            f.write(f'\n')
    else:
        with open(outfile + '_SigOnlyInFemale.txt', 'a') as f:
            f.write(f'{featname}_{valuetype}|{sig_group_effect_female}')
            f.write(f'\n')


        
    # with open(outfile, 'a') as f:
    #     f.write(f'\n')
    #     f.write(f'=============== value: {featname}_{valuetype} ====================\n')
    #     f.write(f'{interaction.to_string()}')
    #     f.write(f'\n')

    #     f.write(f'{group_effect_male.to_string()}')
    #     f.write(f'\n')

    #     f.write(f'{group_effect_female.to_string()}')

    #     f.write(f'\n')
    #     f.write(f'{result.to_string()}')
    #     f.write(f'\n')
    #     f.write(f'\n')
    
    return df    
    

def visualize_feat_boxplot(df, featname, outfile):
    """
    Visualizes the specified feature with respect to group (PD vs HC) and gender (Female vs Male) using a box plot.
    
    Parameters:
    - df: pandas DataFrame containing the data.
    - featname: str, name of the feature to visualize (used for labeling).
    - outfile: str, path to save the visualization.
    """
    # Map gender codes to labels
    df['gender'] = df['gender'].map({'V': 'Female', 'M': 'Male'})
    
    # Verify that 'value' column exists
    if 'value' not in df.columns:
        raise ValueError("The DataFrame does not contain a 'value' column.")
    
    # Set the aesthetic style of the plots
    sns.set(style="whitegrid")
    
    plt.figure(figsize=(8, 6))
    
    # Box Plot using 'value' as the y-axis
    ax = sns.boxplot(
        x='gender',
        y='value',            # Use 'value' instead of featname
        hue='group_id',
        data=df,
        palette='muted',
        showfliers=False     # Optional: hide outliers
    )
    
    plt.title(f'{featname.capitalize()} by Group and Gender')
    plt.xlabel('Gender')
    plt.ylabel(featname.capitalize())
    
    # Add individual data points
    sns.stripplot(
        x='gender',
        y='value',            # Use 'value' instead of featname
        hue='group_id',
        data=df,
        dodge=True,
        color='black',
        alpha=0.5,
        size=3
    )
    
    # Adjust the legend to avoid duplicate labels
    handles, labels = ax.get_legend_handles_labels()
    if handles:
        plt.legend(handles[:2], labels[:2], title='Group')
    
    plt.tight_layout()
    plt.savefig(outfile, dpi=300)
    plt.show()
    
    print(f"{featname.capitalize()} visualization saved to '{outfile}'")


def generate_all_lme(outfile, allfeats, base_folder_path, base_folder_path_unnorm, feats2level, visualize=False, gender_ref='V'):
  
    outfile = (outfile, gender_ref)
    feature_dfs = {}  # Dictionary to store DataFrames for each feature
    
    for featname in allfeats:
        folder = base_folder_path_unnorm if featname == 'energy' else base_folder_path

        if feats2level[featname] == 'frame':
            metadata = load_feat(folder, feature_name=featname)
            df = lme(metadata, outfile, featname=featname, level=feats2level[featname], stats='mean')
            feature_dfs[featname + '_mean'] = df

            df = lme(metadata, outfile, featname=featname, level=feats2level[featname], stats='std')
            # Optionally, process 'std' or other statistics
            feature_dfs[featname + '_std'] = df
            
        elif feats2level[featname] == 'utt':
            metadata = load_feat(folder, feature_name=featname)
            df = lme(metadata, outfile, featname=featname, level=feats2level[featname])
            feature_dfs[featname] = df
        
        elif feats2level[featname] == '3d':
            metadatas = load_feat(folder, feature_name=featname, threeD=True)
            for i, metadata in enumerate(metadatas):
                df = lme(metadata, outfile, featname=f'{featname}_{i+1}', level='frame', stats='mean')
                feature_dfs[f'{featname}_{i}_mean'] = df
                df = lme(metadata, outfile, featname=f'{featname}_{i+1}', level='frame', stats='std')
                feature_dfs[f'{featname}_{i}_std'] = df

    if visualize:
        for featname in feature_dfs.keys():
            visualize_feat_boxplot(feature_dfs[featname], featname, f'./res_lme/{featname}_visualization.png')
            

   
def cmp_gender_all_lme(outfile, allfeats, base_folder_path, base_folder_path_unnorm, feats2level, visualize=False, gender_ref='V'):
  
    outfile = (outfile, gender_ref)
    feature_dfs = {}  # Dictionary to store DataFrames for each feature
    
    for featname in allfeats:
        folder = base_folder_path_unnorm if featname == 'energy' else base_folder_path

        if feats2level[featname] == 'frame':
            metadata = load_feat(folder, feature_name=featname)
            df = lme_cmp_gender(metadata, outfile, featname=featname, level=feats2level[featname], stats='mean')
            feature_dfs[featname + '_mean'] = df

            df = lme_cmp_gender(metadata, outfile, featname=featname, level=feats2level[featname], stats='std')
            # Optionally, process 'std' or other statistics
            feature_dfs[featname + '_std'] = df
            
        elif feats2level[featname] == 'utt':
            metadata = load_feat(folder, feature_name=featname)
            df = lme_cmp_gender(metadata, outfile, featname=featname, level=feats2level[featname])
            feature_dfs[featname] = df
        
        elif feats2level[featname] == '3d':
            metadatas = load_feat(folder, feature_name=featname, threeD=True)
            for i, metadata in enumerate(metadatas):
                df = lme_cmp_gender(metadata, outfile, featname=f'{featname}_{i+1}', level='frame', stats='mean')
                feature_dfs[f'{featname}_{i}_mean'] = df
                df = lme_cmp_gender(metadata, outfile, featname=f'{featname}_{i+1}', level='frame', stats='std')
                feature_dfs[f'{featname}_{i}_std'] = df

    if visualize:
        for featname in feature_dfs.keys():
            visualize_feat_boxplot(feature_dfs[featname], featname, f'./res_lme/{featname}_visualization.png')

     
if __name__ == '__main__':
    base_folder_path_unnorm = Path('/data/storage500/Turntaking/wavs_single_channel_nosil')
    base_folder_path = Path('/data/storage500/Turntaking/wavs_single_channel_normalized_nosil')

    feats2level = {
        'jitter': 'utt',
        'shimmer': 'utt',
        'rp': 'utt',
        'f0': 'frame',
        'energy': 'frame',
        'spec_cent': 'frame',
        'spec_bw': 'frame',
        'spec_rolloff': 'frame',
        'spec_rolloff_min': 'frame',
        'zcr': 'frame',
        'flatness': 'frame',
        'F1': 'frame',
        'F2': 'frame',
        'F3': 'frame',
        'hnr_value': 'utt',
        'contrast': '3d',

    }

    np.set_printoptions(precision=2)
    allfeats_batch1 = ['jitter', 'shimmer', 'rp', 'f0', 'energy']
    allfeats_batch2 = ['F1', 'F2', 'F3', 'zcr', 'flatness', 'hnr_value','spec_cent', 'spec_bw', 'spec_rolloff', 'spec_rolloff_min', 'contrast']
    allfeats = allfeats_batch1 + allfeats_batch2
    
    # allfeats = ['shimmer']
    
    gender_ref = 'M'
    outfile = './res_lme/res_v2_' + gender_ref + '.txt'
    if os.path.exists(outfile):
        os.remove(outfile)
        
    # generate_all_lme(outfile, allfeats, base_folder_path, base_folder_path_unnorm, feats2level, gender_ref=gender_ref)

    outfile_cmp = './res_lme/res_cmp_gender'
    os.system(f"rm {outfile_cmp}*")
    cmp_gender_all_lme(outfile_cmp, allfeats, base_folder_path, base_folder_path_unnorm, feats2level, gender_ref=gender_ref)

        
    



rm: cannot remove './res_lme/res_cmp_gender*': No such file or directory


Processing PictureNaming folder...
Found 1652 npy files
0 files with all 0 values
Processing EarlyLate folder...
Found 3971 npy files
nan in /data/storage500/Turntaking/wavs_single_channel_normalized_nosil/EarlyLate-features/jitter/subj-2120_27_E_kwaad_geloof.wav_1.wav_jitter.npy
All nan value in /data/storage500/Turntaking/wavs_single_channel_normalized_nosil/EarlyLate-features/jitter/subj-2120_27_E_kwaad_geloof.wav_1.wav_jitter.npy
0 files with all 0 values
Processing BoundaryTone folder...
Found 5026 npy files
0 files with all 0 values
warning 'age' has 156 NaN, has been deleted
Linear mixed model fit by REML [’lmerMod’]
Formula: value~age+experiment+group_id*gender+(1|subject_id)+(1|item)

Family: gaussian	 Inference: parametric

Number of observations: 7762	 Groups: {'item': 140.0, 'subject_id': 51.0}

Log-likelihood: 26469.480 	 AIC: -52918.959

Random effects:

                   Name  Var    Std
item        (Intercept)  0.0  0.004
subject_id  (Intercept)  0.0  0.004
Residual   

/tmp/ipykernel_7952/3939088053.py:151: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  df['value'] = df['value'].apply(lambda x: float(x))


Linear mixed model fit by REML [’lmerMod’]
Formula: value~age+experiment+group_id*gender+(1|subject_id)+(1|item)

Family: gaussian	 Inference: parametric

Number of observations: 7763	 Groups: {'item': 140.0, 'subject_id': 51.0}

Log-likelihood: -9561.833 	 AIC: 19143.667

Random effects:

                   Name    Var    Std
item        (Intercept)  0.222  0.472
subject_id  (Intercept)  0.098  0.313
Residual                 0.643  0.802

No random effect correlations specified

Fixed effects:

Linear mixed model fit by REML [’lmerMod’]
Formula: value~age+experiment+group_id*gender+(1|subject_id)+(1|item)

Family: gaussian	 Inference: parametric

Number of observations: 7763	 Groups: {'item': 140.0, 'subject_id': 51.0}

Log-likelihood: -9561.833 	 AIC: 19143.667

Random effects:

                   Name    Var    Std
item        (Intercept)  0.222  0.472
subject_id  (Intercept)  0.098  0.313
Residual                 0.643  0.802

No random effect correlations specified

Fixed effects:

RRuntimeError: 